In [1]:
import sys
import numpy as np
import wfsim
print(wfsim.__version__)

del sys.path[5]
sys.path.insert(0,'/home/pkavrigin/epix') # 'fast_sim_test' branch

import epix
print(epix.__version__)

import straxen
from epix.simulator.fast_simulator import StraxSimulator

/dali/lgrandi/strax/straxen/straxen/rucio.py:29: UserWarning: No installation of rucio-clients found. Can't use rucio remote backend.
  warnings.warn("No installation of rucio-clients found. Can't use rucio remote backend.")


0.5.4
Using nestpy version 1.3.0
0.1.2


In [2]:
raw_data_dir='/dali/lgrandi/pkavrigin/'
raw_data_filename='ambe_tut_cw5900mm.root'

fast_sim_output_path= '/home/pkavrigin/tmp/strax/'

In [3]:
st = straxen.contexts.xenonnt_simulation(output_folder=fast_sim_output_path)
run_id = '1'

#fax_config='fax_config_nt_design.json'
fax_config='fax_config_nt_low_field.json'

# epix config
epix_args={'path':raw_data_dir,
           'file_name':raw_data_filename, 
           'debug':True,           
           'entry_start':0,
           'entry_stop':1000,
           'cut_by_eventid':False,
           'micro_separation_time':10.0,
           'micro_separation':0.00001,
           'tag_cluster_by':'time',
           'max_delay':1e7,
           'source_rate':-1}

# configuration files; to be moved into the plugin
configuration_files={'nv_pmt_qe':'nveto_pmt_qe.json',
                     'photon_area_distribution':'XENONnT_spe_distributions_20210305.csv',
                     's1_relative_lce_map':straxen.pax_file('XENON1T_s1_xyz_lce_true_kr83m_SR1_pax-680_fdc-3d_v0.json'),
                     's2_xy_correction_map':straxen.pax_file('XENON1T_s2_xy_ly_SR1_v2.2.json'),}

st.config.update(dict(fax_config=fax_config,
                      g4_file=raw_data_dir+raw_data_filename,
                      epix_config=epix_args,
                      configuration_files=configuration_files,
                      fax_config_overrides= {'detector':'XENONnT',
                                             "enable_field_dependencies": {
                                             "survival_probability_map": False,
                                             "drift_speed_map": False,
                                             "diffusion_longitudinal_map": False,
                                             "diffusion_radial_map": False,
                                             "diffusion_azimuthal_map": False}}))

In [4]:
st.register(StraxSimulator)

epix.simulator.fast_simulator.StraxSimulator

In [5]:
st.make(run_id,'events_tpc')

epix configuration:  {'path': '/dali/lgrandi/pkavrigin/', 'file_name': 'ambe_tut_cw5900mm.root', 'debug': True, 'entry_start': 0, 'entry_stop': 1000, 'cut_by_eventid': False, 'micro_separation_time': 10.0, 'micro_separation': 1e-05, 'tag_cluster_by': 'time', 'max_delay': 10000000.0, 'source_rate': -1, 'detector_config': [<epix.detector_volumes.SensitiveVolume object at 0x7f3bf87277f0>, <epix.detector_volumes.SensitiveVolume object at 0x7f3bf8727ee0>], 'outer_cylinder': {'max_z': 7.3936, 'min_z': -154.6555, 'max_r': 66.4}}
Total entries in input file = 25937
Starting to read from output file entry 0
Ending read in at output file entry 1000
It took 9.0910 sec to load data.
Finding clusters of interactions with a dr = 1e-05 mm and dt = 10.0 ns
It took 5.5375 sec to find clusters.
It took 6.3715 sec to merge clusters.
Removing clusters not in volumes: TPC BelowCathode
Number of clusters before: 60207
Number of clusters after: 58444
Assigning electric field to clusters
Generating photons an

/dali/lgrandi/strax/miniconda3/envs/strax/lib/python3.8/site-packages/numpy/lib/function_base.py:2197: RuntimeWarning: divide by zero encountered in func (vectorized)
  outputs = ufunc(*inputs)
/dali/lgrandi/strax/miniconda3/envs/strax/lib/python3.8/site-packages/numpy/lib/function_base.py:2197: RuntimeWarning: overflow encountered in func (vectorized)
  outputs = ufunc(*inputs)
/dali/lgrandi/strax/miniconda3/envs/strax/lib/python3.8/site-packages/numpy/lib/function_base.py:2197: RuntimeWarning: invalid value encountered in func (vectorized)
  outputs = ufunc(*inputs)


It took 4.5148 sec to get quanta.
Clean event separation
Min. S2 amp BEFORE MACRO-CLUSTERING: 1


Exception in thread CRITICAL:ThreadedMailboxProcessor:Target Mailbox (events_tpc) killed, exception <class 'strax.mailbox.MailboxKilled'>, message (<class 'ValueError'>, ValueError('n < 0'), <traceback object at 0x7f3bf87be7c0>)
divide_outputs:events_tpc:
Traceback (most recent call last):
  File "/dali/lgrandi/strax/miniconda3/envs/strax/lib/python3.8/threading.py", line 932, in _bootstrap_inner
    self.run()
  File "/dali/lgrandi/strax/miniconda3/envs/strax/lib/python3.8/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/dali/lgrandi/strax/strax/strax/mailbox.py", line 267, in _send_from
    self.kill_from_exception(e)
  File "/dali/lgrandi/strax/strax/strax/mailbox.py", line 186, in kill_from_exception
    raise e
  File "/dali/lgrandi/strax/strax/strax/mailbox.py", line 254, in _send_from
    x = next(iterable)
  File "/dali/lgrandi/strax/strax/strax/plugin.py", line 394, in iter
    yield self.do_compute(chunk_i=chunk_i, **inputs_merged)
  File

Min. S2 amp AFTER MACRO-CLUSTERING: -3


ValueError: n < 0

In [ ]:
arr_events = st.get_array(run_id, 'events_tpc')
arr_events.dtype